In [12]:
!pip install gensim fasttext
!pip install python==3.8
!pip install torch
!pip install pandas
!pip install scikit-learn
!pip install transformer
!pip install datasets
!pip install accelerate==0.26.0

ERROR: Could not find a version that satisfies the requirement python==3.8 (from versions: none)
ERROR: No matching distribution found for python==3.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    F

In [7]:
# ! unzip TP_ISD2020.zip -d data
# ! unzip scripts.zip
# ! rm -r __MACOSX/

In [24]:
def load_corpus(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        corpus = [line.strip().split() for line in f]
    return corpus

# Chemins des fichiers
medical_corpus_path = "data/QUAERO_FrenchMed/QUAERO_FrenchMed_traindev.ospl"
press_corpus_path = "data/QUAERO_FrenchPress/QUAERO_FrenchPress_traindev.ospl"

# Charger les corpus
medical_corpus = load_corpus(medical_corpus_path)
press_corpus = load_corpus(press_corpus_path)

candidate_words = ["patient", "traitement", "maladie", "solution", "jaune"]

In [20]:
from gensim.models import Word2Vec
import fasttext
import fasttext.util
from scipy.spatial.distance import cosine

def train_word2vec(corpus, sg, save_path, epochs=5):
    # Entraîner le modèle
    model = Word2Vec(
        sentences=corpus,
        vector_size=100,  # Taille des vecteurs
        window=5,         # Taille du contexte
        min_count=1,      # Fréquence minimale des mots
        sg=sg,            # 0 = CBOW, 1 = Skipgram
        workers=4,         # Nombre de threads
        epochs=epochs
    )
    # Sauvegarder le modèle
    model.save('model/' + save_path)
    print(f"Modèle Word2Vec sauvegardé dans {save_path}")

def train_fasttext(corpus_path, save_path, epochs=5):
    # Entraîner le modèle FastText (CBOW)
    model = fasttext.train_unsupervised(
        input=corpus_path,
        model='cbow',  # CBOW
        dim=100,       # Taille des vecteurs
        minCount=1,     # Fréquence minimale des mots
        epoch=epochs
    )
    # Sauvegarder le modèle
    model.save_model('model/' + save_path)
    print(f"Modèle FastText sauvegardé dans {save_path}")

def gensim_most_similar(model, word, topn=10):
    """
    Trouve les mots les plus proches d'un mot donné avec Gensim.
    :param model: Modèle Word2Vec ou FastText
    :param word: Mot de départ
    :param topn: Nombre de mots similaires à retourner
    """
    try:
        return model.wv.most_similar(word, topn=topn)
    except KeyError:
        return f"Le mot '{word}' n'est pas dans le vocabulaire du modèle."

def scipy_most_similar(model, word, topn=10):
    """
    Trouve les mots les plus proches d'un mot donné avec scipy.
    :param model: Modèle Word2Vec ou FastText
    :param word: Mot de départ
    :param topn: Nombre de mots similaires à retourner
    """
    try:
        word_vector = model.wv[word]
    except KeyError:
        return f"Le mot '{word}' n'est pas dans le vocabulaire du modèle."

    # Calculer la similarité cosinus avec tous les mots du vocabulaire
    similarities = []
    for other_word in model.wv.index_to_key:
        similarity = 1 - cosine(word_vector, model.wv[other_word])
        similarities.append((other_word, similarity))

    # Trier par similarité décroissante et retourner les topn
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:topn]


In [23]:
# Entraîner et sauvegarder les modèles Word2Vec pour choisir le meilleur nombre d'epoch
train_word2vec(medical_corpus, sg=0, save_path="word2vec_cbow_med_train1.model", epochs=1)
train_word2vec(medical_corpus, sg=0, save_path="word2vec_cbow_med_train5.model", epochs=5)
train_word2vec(medical_corpus, sg=0, save_path="word2vec_cbow_med_train20.model", epochs=20)
train_word2vec(medical_corpus, sg=0, save_path="word2vec_cbow_med_train100.model", epochs=100)

Modèle Word2Vec sauvegardé dans word2vec_cbow_med_train1.model
Modèle Word2Vec sauvegardé dans word2vec_cbow_med_train5.model
Modèle Word2Vec sauvegardé dans word2vec_cbow_med_train20.model
Modèle Word2Vec sauvegardé dans word2vec_cbow_med_train100.model


In [31]:
word2vec_cbow_med_train1 = Word2Vec.load("model/word2vec_cbow_med_train1.model")
word2vec_cbow_med_train5 = Word2Vec.load("model/word2vec_cbow_med_train5.model")
word2vec_cbow_med_train20 = Word2Vec.load("model/word2vec_cbow_med_train20.model")
word2vec_cbow_med_train100 = Word2Vec.load("model/word2vec_cbow_med_train100.model")

for word in candidate_words:
    print(f"\nRésultats pour le mot '{word}' :")
    print("Word2Vec CBOW MED(1):", gensim_most_similar(word2vec_cbow_med_train1, word))
    print("Word2Vec CBOW MED(5):", gensim_most_similar(word2vec_cbow_med_train5, word))
    print("Word2Vec CBOW MED(20):", gensim_most_similar(word2vec_cbow_med_train20, word))
    print("Word2Vec CBOW MED(100):", gensim_most_similar(word2vec_cbow_med_train100, word))

# 100 epochs semble des résultats légèrement meilleurs que 20 epochs mais
# pour réduire le temps de calcul je vais utiliser 20 epochs.


Résultats pour le mot 'patient' :
Word2Vec CBOW MED(1): [('pendant', 0.638522744178772), ('Eli', 0.6373327374458313), ('La', 0.6337447762489319), ('sous', 0.6329166293144226), ('dose', 0.6322556138038635), ('ET', 0.6320965886116028), ("'", 0.6317471861839294), ('.', 0.6304958462715149), ('une', 0.6267744898796082), ('le', 0.6260660886764526)]
Word2Vec CBOW MED(5): [('Le', 0.9993764162063599), ('le', 0.9993761777877808), ('La', 0.9993587136268616), ('les', 0.999343752861023), ('par', 0.9993258118629456), ('du', 0.9993196129798889), ('avec', 0.999306321144104), ('plus', 0.9993047714233398), ('en', 0.9993032813072205), ('des', 0.9993005394935608)]
Word2Vec CBOW MED(20): [('qui', 0.9822223782539368), ('agir', 0.9793098568916321), ('devra', 0.9766798615455627), ('délai', 0.9753844738006592), ('assurer', 0.9748953580856323), ('ils', 0.9743163585662842), ('arrêter', 0.9728955030441284), ('car', 0.9710395932197571), ('ce', 0.9707961678504944), ('médicament', 0.9705470204353333)]
Word2Vec CBOW

In [34]:
# Entraîner et sauvegarder les modèles FastText
train_fasttext(medical_corpus_path, "fasttext_cbow_med_1.bin", epochs=1)
train_fasttext(medical_corpus_path, "fasttext_cbow_med_5.bin", epochs=5)
train_fasttext(medical_corpus_path, "fasttext_cbow_med_20.bin", epochs=20)
train_fasttext(medical_corpus_path, "fasttext_cbow_med_100.bin", epochs=100)

Modèle FastText sauvegardé dans fasttext_cbow_med_1.bin
Modèle FastText sauvegardé dans fasttext_cbow_med_5.bin
Modèle FastText sauvegardé dans fasttext_cbow_med_20.bin
Modèle FastText sauvegardé dans fasttext_cbow_med_100.bin


In [35]:
fasttext_cbow_med_1 = fasttext.load_model("model/fasttext_cbow_med_1.bin")
fasttext_cbow_med_5 = fasttext.load_model("model/fasttext_cbow_med_5.bin")
fasttext_cbow_med_20 = fasttext.load_model("model/fasttext_cbow_med_20.bin")
fasttext_cbow_med_100 = fasttext.load_model("model/fasttext_cbow_med_100.bin")

for word in candidate_words:
    print(f"\nRésultats pour le mot '{word}' :")
    print(fasttext_cbow_med_1.get_nearest_neighbors(word))
    print(fasttext_cbow_med_5.get_nearest_neighbors(word))
    print(fasttext_cbow_med_20.get_nearest_neighbors(word))
    print(fasttext_cbow_med_100.get_nearest_neighbors(word))

# 10 epochs semble avoir les meilleurs resultats


Résultats pour le mot 'patient' :
[(0.9402261972427368, 'patiente'), (0.9213680028915405, 'patients'), (0.8477667570114136, 'entrainement'), (0.8382683992385864, 'appartient'), (0.8129658102989197, 'département'), (0.8122319579124451, 'Traitements'), (0.8105729222297668, 'inventaires'), (0.8060504794120789, 'pigmentaires'), (0.8058280944824219, 'conscient'), (0.8043614029884338, 'Traitement')]
[(1.000000238418579, 'ventriculaires'), (1.000000238418579, 'avacement'), (1.000000238418579, 'cohérent'), (1.000000238418579, 'isolement'), (1.0000001192092896, 'comment'), (1.0000001192092896, 'retentissement'), (1.0000001192092896, 'spécifiquement'), (1.0000001192092896, 'Allaitement'), (1.0, 'intravasculaires'), (1.0, 'paramétriales')]
[(0.9993148446083069, 'entrainement'), (0.999184787273407, 'Accompagnement'), (0.9991169571876526, 'récemment'), (0.9990652799606323, 'comportement'), (0.9990555047988892, 'soulagement'), (0.9990339875221252, 'glissement'), (0.9990159869194031, 'socialement'),

In [36]:
# Entraîner et sauvegarder les modèles Word2Vec
train_word2vec(medical_corpus, sg=0, save_path="word2vec_cbow_med.model", epochs=20)
train_word2vec(medical_corpus, sg=1, save_path="word2vec_skipgram_med.model", epochs=20)
train_word2vec(press_corpus, sg=0, save_path="word2vec_cbow_press.model", epochs=20)
train_word2vec(press_corpus, sg=1, save_path="word2vec_skipgram_press.model", epochs=20)
train_fasttext(medical_corpus_path, "fasttext_cbow_med.bin", epochs=10)
train_fasttext(press_corpus_path, "fasttext_cbow_press.bin", epochs=10)

Modèle Word2Vec sauvegardé dans word2vec_cbow_med.model
Modèle Word2Vec sauvegardé dans word2vec_skipgram_med.model
Modèle Word2Vec sauvegardé dans word2vec_cbow_press.model
Modèle Word2Vec sauvegardé dans word2vec_skipgram_press.model
Modèle FastText sauvegardé dans fasttext_cbow_med.bin
Modèle FastText sauvegardé dans fasttext_cbow_press.bin


In [39]:
word2vec_cbow_med = Word2Vec.load("model/word2vec_cbow_med.model")
word2vec_skipgram_med = Word2Vec.load("model/word2vec_skipgram_med.model")
word2vec_cbow_press = Word2Vec.load("model/word2vec_cbow_press.model")
word2vec_skipgram_press = Word2Vec.load("model/word2vec_skipgram_press.model")

fasttext_cbow_med = fasttext.load_model("model/fasttext_cbow_med.bin")
fasttext_cbow_press = fasttext.load_model("model/fasttext_cbow_press.bin")

for word in candidate_words:
    print(f"\nRésultats pour le mot '{word}' :")
    print("Word2Vec CBOW (medical):", gensim_most_similar(word2vec_cbow_med, word))
    print("Word2Vec Skipgram (medical):", gensim_most_similar(word2vec_skipgram_med, word))
    print("Word2Vec CBOW (non-medical):", gensim_most_similar(word2vec_cbow_press, word))
    print("Word2Vec Skipgram (non-medical):", gensim_most_similar(word2vec_skipgram_press, word))
    print("FastText CBOW (medical):", fasttext_cbow_med.get_nearest_neighbors(word))
    print("FastText CBOW (non-medical):", fasttext_cbow_press.get_nearest_neighbors(word))


Résultats pour le mot 'patient' :
Word2Vec CBOW (medical): [('qui', 0.9862404465675354), ('délai', 0.9780006408691406), ('carte', 0.9778679013252258), ('particulièrement', 0.9759427905082703), ('aptitude', 0.9758378267288208), ('risque', 0.9757390022277832), ('modifier', 0.9745245575904846), ('devra', 0.9735966324806213), ('risques', 0.9726468324661255), ('cette', 0.972396194934845)]
Word2Vec Skipgram (medical): [('carte', 0.8953224420547485), ('Montrez', 0.8711549639701843), ('allergique', 0.8686885833740234), ('avoir', 0.8642374277114868), ('alerte', 0.8627194762229919), ('aptitude', 0.8571023941040039), ('examiner', 0.8545088171958923), ('répondu', 0.8535452485084534), ('atteint', 0.8530545830726624), ('survenir', 0.8459311723709106)]
Word2Vec CBOW (non-medical): [('concessionnaire', 0.6684316396713257), ('morceau', 0.6401335000991821), ('lac', 0.6392399668693542), ('colis', 0.6310966610908508), ('collier', 0.6302106380462646), ('emprunt', 0.6226995587348938), ('troupeau', 0.621496